In [1]:
import pandas as pd
import numpy as np

/var/folders/n5/f6xjtlr526zbr86__yns243m0000gn/T/ipykernel_78664/2162656668.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Data Preprocessing

### 1. Book-Crossing Dataset

In [2]:
# Reading the CSV files and skipping the bad lines
books = pd.read_csv('/Users/sanjeevani1109/Desktop/Book_Recommendation_System/Book crossing /BX-Books.csv', sep=';',on_bad_lines = 'skip', encoding='latin-1')
ratings = pd.read_csv('/Users/sanjeevani1109/Desktop/Book_Recommendation_System/Book crossing /BX-Book-Ratings.csv', sep=';',on_bad_lines = 'skip', encoding='latin-1')
users = pd.read_csv('/Users/sanjeevani1109/Desktop/Book_Recommendation_System/Book crossing /BX-Users.csv',sep=';',on_bad_lines = 'skip', encoding='latin-1')

/var/folders/n5/f6xjtlr526zbr86__yns243m0000gn/T/ipykernel_78664/3698374196.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('/Users/sanjeevani1109/Desktop/Book_Recommendation_System/Book crossing /BX-Books.csv', sep=';',on_bad_lines = 'skip', encoding='latin-1')


In [3]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


### 1.1 Books Table

In [4]:
#Renaming the columns
books.rename(columns={'Book-Title': 'Title', 'Book-Author': 'Author', 'Year-Of-Publication': 'Year'}, inplace=True)

In [5]:
#Dropping the URL columns for the Books table
books.drop(['Image-URL-S', 'Image-URL-M', 'Image-URL-L'], axis=1, inplace=True)

In [6]:
books.loc[books['Author'].isnull(), 'Author'] = "Others"

In [7]:
books.loc[books['Publisher'].isnull(), 'Publisher'] = "Others"

In [8]:
books = books.drop_duplicates()

In [9]:
books.at[209538, 'Publisher'] = 'DK Publishing Inc'
books.at[209538, 'Year'] = 2000
books.at[209538, 'Title'] = 'DK Readers: Creating the X-Men, How It All Began (Level 4: Proficient Readers)'
books.at[209538, 'Author'] = 'Michael Teitelbaum'

In [10]:
books.at[221678, 'Publisher'] = 'DK Publishing Inc'
books.at[221678, 'Year'] = 2000
books.at[221678, 'Title'] = 'DK Readers: Creating the X-Men, How Comic Books Come to Life (Level 4: Proficient Readers)'
books.at[221678, 'Author'] = 'James Buckley'

In [11]:
books.at[220731, 'Publisher'] = 'Gallimard'
books.at[220731, 'Year'] = 2003
books.at[220731, 'Title'] = 'Peuple du ciel - Suivi de Les bergers '
books.at[220731, 'Author'] = 'Jean-Marie Gustave Le ClÃ?Â©zio'

In [12]:
#converting year to integer values
books['Year'] = pd.to_numeric(books['Year'], errors='coerce')

In [13]:
count = books['Year'].value_counts()
most_commen_year = count[count == count.max()].index.tolist()[0]

In [14]:
#Replacing all values for Publishing years greater than 2004 or equal to 0 with the maximum year
books.loc[books['Year'] > 2004, 'Year'] = most_commen_year

In [15]:
books.loc[books['Year'] == 0, 'Year'] = most_commen_year

### 1.2 Users Table

In [16]:
#Renaming the columns
users.rename(columns={'User-ID': 'UserID'}, inplace=True)

#### Ages that have values nan, less than 8, and more than 90 are invalid 

In [17]:
age_range = users[users['Age'] <= 90] 
age_range = age_range[age_range['Age'] >= 8]
mean = age_range['Age'].mean().round()   

In [18]:
users.loc[(users['Age'] > 90) | (users['Age'] < 8),'Age'] = mean
users['Age']=users['Age'].fillna(mean).astype(int)

### 1.3 Ratings Table

In [19]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [20]:
#Renaming the columns
ratings.rename(columns={'User-ID': 'UserID', 'Book-Rating': 'BookRating'}, inplace=True)

In [21]:
ratings = ratings.drop(["UserID"], axis=1)

In [22]:
bx_userrate = ratings.groupby(['ISBN'], as_index=False)['BookRating'].mean().round(0)
bx_userrate.value_counts()

ISBN         BookRating
 0330299891  3.0           1
0875961096   0.0           1
0875961010   0.0           1
087595247X   0.0           1
0875952372   10.0          1
                          ..
0451124847   0.0           1
0451124693   0.0           1
0451124685   2.0           1
0451124634   0.0           1
Ô½crosoft    7.0           1
Name: count, Length: 340556, dtype: int64

In [23]:
#Scaling the ratings to from a scale of 1-10 to 1-5
scaled_ratings = (bx_userrate['BookRating'] - 1) * (5 / 9) + 1
bx_userrate['Scaled Rating'] = np.clip(round(scaled_ratings), 1, 5)
bx_userrate['Scaled Rating'].unique()

array([2., 1., 3., 5., 4.])

In [24]:
bx_userrate['Scaled Rating'] = bx_userrate['Scaled Rating'].astype(int)
bx_userrate.head()

,ISBN,BookRating,Scaled Rating
0,0330299891,3.0,2
1,0375404120,2.0,2
2,0586045007,0.0,1
3,9022906116,4.0,3
4,9032803328,0.0,1


### 1.4 Merging Books, Users, and Ratings tables

In [25]:
bx = pd.merge(books, bx_userrate, on='ISBN')

### 2. Goodbooks-10k Dataset

In [98]:
goodbooks = pd.read_csv('/Users/sanjeevani1109/Desktop/Book_Recommendation_System/goodreads-10k/books.csv', sep=',',on_bad_lines = 'skip', encoding='latin-1')
goodratings = pd.read_csv('/Users/sanjeevani1109/Desktop/Book_Recommendation_System/goodreads-10k/ratings.csv', sep=',',on_bad_lines = 'skip', encoding='latin-1')
goodbooks_tags = pd.read_csv('/Users/sanjeevani1109/Desktop/Book_Recommendation_System/goodreads-10k/book_tags.csv',sep=',',on_bad_lines = 'skip', encoding='latin-1')
goodtags = pd.read_csv('/Users/sanjeevani1109/Desktop/Book_Recommendation_System/goodreads-10k/tags.csv',sep=',',on_bad_lines = 'skip', encoding='latin-1')
good_toread = pd.read_csv('/Users/sanjeevani1109/Desktop/Book_Recommendation_System/goodreads-10k/to_read.csv',sep=',',on_bad_lines = 'skip', encoding='latin-1')

### 2.1 Books Table

In [99]:
#Dropping the URL columns for the Books table
goodbooks.drop(['image_url', 'small_image_url'], axis=1, inplace=True)

In [100]:
#Renaming the columns
goodbooks.rename(columns={'book_id': 'BookID', 'isbn': 'ISBN', 'authors': 'Author', 'original_publication_year': 'Year', 'original_title': 'Title', 'average_rating': 'Avgrating'}, inplace=True)

In [101]:
average_year = goodbooks.loc[goodbooks['Year'] >= 1800, 'Year'].mean().round()
average_year

1994.0

In [102]:
goodbooks.loc[goodbooks['Year'] == 0, 'Year'] = average_year
goodbooks.loc[:, 'Year'] = goodbooks['Year'].fillna(average_year)
goodbooks.loc[goodbooks['Year'] < 1800, 'Year'] = average_year

In [103]:
goodbooks.dropna(subset=['Title'], inplace=True)

In [104]:
goodbooks.shape

(9415, 21)

In [105]:
goodbooks['Year'] = goodbooks['Year'].astype(int)

In [106]:
#creating a new data frame only with the relevant columns
selected_cols = ['id', 'BookID', 'Author', 'Year', 'Title', 'Avgrating', 'ratings_count']
new_goodbooks = goodbooks[selected_cols].copy()

In [114]:
new_goodbooks.shape

(9415, 7)

### 2.2 Ratings Table

In [115]:
#Renaming the columns
goodratings.rename(columns={'book_id': 'BookID', 'user_id': 'UserID', 'rating': 'Rating'}, inplace=True)

In [116]:
user_rate = goodratings.groupby(['BookID'], as_index=False)['Rating'].mean().round(0)

user_rate.value_counts()

BookID  Rating
1       4.0       1
6671    4.0       1
6664    4.0       1
6665    4.0       1
6666    4.0       1
                 ..
3334    4.0       1
3335    4.0       1
3336    4.0       1
3337    4.0       1
10000   4.0       1
Name: count, Length: 10000, dtype: int64

In [118]:
user_rate.shape

(10000, 2)

In [119]:
#Renaming the columns
user_rate.rename(columns={'Rating': 'Mean Rating'}, inplace=True)

user_rate.value_counts()

BookID  Mean Rating
1       4.0            1
6671    4.0            1
6664    4.0            1
6665    4.0            1
6666    4.0            1
                      ..
3334    4.0            1
3335    4.0            1
3336    4.0            1
3337    4.0            1
10000   4.0            1
Name: count, Length: 10000, dtype: int64

In [120]:
user_rate['Mean Rating'] = user_rate['Mean Rating'].astype(int)

In [121]:
user_rate.shape

(10000, 2)

### 2.6 Merging the Books and Ratings tables

In [122]:
new_goodbooks

,id,BookID,Author,Year,Title,Avgrating,ratings_count
0,1,2767052,Suzanne Collins,2008,The Hunger Games,4.34,4780653
1,2,3,"J.K. Rowling, Mary GrandPrÃ©",1997,Harry Potter and the Philosopher's Stone,4.44,4602479
2,3,41865,Stephenie Meyer,2005,Twilight,3.57,3866839
3,4,2657,Harper Lee,1960,To Kill a Mockingbird,4.25,3198671
4,5,4671,F. Scott Fitzgerald,1925,The Great Gatsby,3.89,2683664
...,...,...,...,...,...,...,...
9995,9996,7130616,Ilona Andrews,2010,Bayou Moon,4.09,17204
9996,9997,208324,Robert A. Caro,1990,Means of Ascent,4.25,12582
9997,9998,77431,Patrick O'Brian,1977,The Mauritius Command,4.35,9421
9998,9999,8565083,Peggy Orenstein,2011,Cinderella Ate My Daughter: Dispatches from th...,3.65,11279


In [128]:
user_rate = user_rate.rename(columns={"BookID":"id"})

In [129]:
user_rate.head()

,id,Mean Rating
0,1,4
1,2,4
2,3,3
3,4,4
4,5,4


In [130]:
gbx = pd.merge(new_goodbooks, user_rate, on='id')

In [131]:
gbx.shape

(9415, 8)

In [134]:
gbx.head()

,id,BookID,Author,Year,Title,Avgrating,ratings_count,Mean Rating
0,1,2767052,Suzanne Collins,2008,The Hunger Games,4.34,4780653,4
1,2,3,"J.K. Rowling, Mary GrandPrÃ©",1997,Harry Potter and the Philosopher's Stone,4.44,4602479,4
2,3,41865,Stephenie Meyer,2005,Twilight,3.57,3866839,3
3,4,2657,Harper Lee,1960,To Kill a Mockingbird,4.25,3198671,4
4,5,4671,F. Scott Fitzgerald,1925,The Great Gatsby,3.89,2683664,4


## NEW

In [135]:
import nltk #using the nltk library for tokenization and lemmatization
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

import pandas as pd

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sanjeevani1109/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sanjeevani1109/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/sanjeevani1109/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sanjeevani1109/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [136]:
#importing necessary libraries
import string
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [137]:
#Function to clean text data
def clean_text(text):
    text = text.lower()     #Converting the text to lowercase
    text = re.sub(r'\d+', ' ', text)    # Removing digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Removing special characters
    text = text.translate(str.maketrans('', '', string.punctuation))    #Removing punctuation     
    tokens = word_tokenize(text) #Tokenization of the text
    non_stopwords = []  #Creating an empty list to store the non-stopwords
    stop_words = set(stopwords.words('english')) #Initializing the NLTK English stopwords
    #Lemmatization of the words/tokens
    lemmatizer = WordNetLemmatizer()    #Initialization of the NLTK WordNet Lemmatizer

    #Iterating through the list of tokens, lemmatizing them, and adding it to the list of non-stopwords
    for token in tokens:
        if token not in stop_words:
            lemma = lemmatizer.lemmatize(token) #Performing lemmatization
            non_stopwords.append(lemma)  # Appending the stemmed token back to the list of non-stopwords

    # Joining the tokens/words from the list of non-stopwords into a string
    filtered_text = ' '.join(non_stopwords)
    return filtered_text


In [138]:
# Apply the text_clean function to the 'Title' column of the first data frame 'bx' i.e., the Book Crossing dataset's final data frame
bx['cleaned_title'] = bx['Title'].apply(clean_text)

In [139]:
# Show the first few rows of the cleaned column for the first DataFrame 'bx'
print("First DataFrame - bx:")
print(bx[['Title', 'cleaned_title']].head())

First DataFrame - bx:
                                               Title  \
0                                Classical Mythology   
1                                       Clara Callan   
2                               Decision in Normandy   
3  Flu: The Story of the Great Influenza Pandemic...   
4                             The Mummies of Urumchi   

                                       cleaned_title  
0                                classical mythology  
1                                       clara callan  
2                                  decision normandy  
3  flu story great influenza pandemic search viru...  
4                                      mummy urumchi  


In [140]:
# Apply the text_clean function to the 'Title' column of the second data frame 'gbx' i.e., the Goodbooks-10k dataset's final data frame
gbx['cleaned_title'] = gbx['Title'].apply(clean_text)

In [141]:
# Show the first few rows of the cleaned column for the second DataFrame 'gbx'
print("\nSecond DataFrame - gbx:")
print(gbx[['Title', 'cleaned_title']].head())


Second DataFrame - gbx:
                                      Title                   cleaned_title
0                          The Hunger Games                     hunger game
1  Harry Potter and the Philosopher's Stone  harry potter philosopher stone
2                                  Twilight                        twilight
3                     To Kill a Mockingbird                kill mockingbird
4                          The Great Gatsby                    great gatsby


### Cleaning 'Author' columns

In [142]:
# Apply the clean_text function to the 'Author' column of the first data frame 'bx'
bx['cleaned_author'] = bx['Author'].apply(clean_text)

In [143]:
# Show the first few rows of the cleaned column for the first DataFrame 'bx'
print("First DataFrame - bx:")
print(bx[['Author', 'cleaned_author']].head())

First DataFrame - bx:
                 Author        cleaned_author
0    Mark P. O. Morford        mark p morford
1  Richard Bruce Wright  richard bruce wright
2          Carlo D'Este           carlo deste
3      Gina Bari Kolata      gina bari kolata
4       E. J. W. Barber          e j w barber


In [144]:
# Apply the clean_text function to the 'Author' column of the second data frame 'gbx'
gbx['cleaned_author'] = gbx['Author'].apply(clean_text)

In [145]:
# Show the first few rows of the cleaned column for the second DataFrame 'gbx'
print("\nSecond DataFrame - gbx:")
print(gbx[['Author', 'cleaned_author']].head())


Second DataFrame - gbx:
                         Author           cleaned_author
0               Suzanne Collins          suzanne collins
1  J.K. Rowling, Mary GrandPrÃ©  jk rowling mary grandpr
2               Stephenie Meyer          stephenie meyer
3                    Harper Lee               harper lee
4           F. Scott Fitzgerald       f scott fitzgerald


In [146]:
bx.head()

,Title,Author,Year,Scaled Rating,cleaned_title,cleaned_author,combined
0,Classical Mythology,Mark P. O. Morford,2002,1,classical mythology,mark p morford,classical mythology mark p morford
1,Clara Callan,Richard Bruce Wright,2001,3,clara callan,richard bruce wright,clara callan richard bruce wright
2,Decision in Normandy,Carlo D'Este,1991,3,decision normandy,carlo deste,decision normandy carlo deste
3,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,3,flu story great influenza pandemic search viru...,gina bari kolata,flu story great influenza pandemic search viru...
4,The Mummies of Urumchi,E. J. W. Barber,1999,1,mummy urumchi,e j w barber,mummy urumchi e j w barber


In [147]:
bxcolumns_to_copy = ['ISBN', 'Publisher', 'BookRating']

In [148]:
bx.drop(columns=bxcolumns_to_copy, inplace=True)
bx.head()

KeyError: "['ISBN', 'Publisher', 'BookRating'] not found in axis"

In [149]:
gbx.head()

,id,BookID,Author,Year,Title,Avgrating,ratings_count,Mean Rating,cleaned_title,cleaned_author
0,1,2767052,Suzanne Collins,2008,The Hunger Games,4.34,4780653,4,hunger game,suzanne collins
1,2,3,"J.K. Rowling, Mary GrandPrÃ©",1997,Harry Potter and the Philosopher's Stone,4.44,4602479,4,harry potter philosopher stone,jk rowling mary grandpr
2,3,41865,Stephenie Meyer,2005,Twilight,3.57,3866839,3,twilight,stephenie meyer
3,4,2657,Harper Lee,1960,To Kill a Mockingbird,4.25,3198671,4,kill mockingbird,harper lee
4,5,4671,F. Scott Fitzgerald,1925,The Great Gatsby,3.89,2683664,4,great gatsby,f scott fitzgerald


In [150]:
newgbxcolumns_to_copy = ['id','BookID', 'Avgrating', 'ratings_count']

In [151]:
gbx.drop(columns=newgbxcolumns_to_copy, inplace=True)
gbx.head()

,Author,Year,Title,Mean Rating,cleaned_title,cleaned_author
0,Suzanne Collins,2008,The Hunger Games,4,hunger game,suzanne collins
1,"J.K. Rowling, Mary GrandPrÃ©",1997,Harry Potter and the Philosopher's Stone,4,harry potter philosopher stone,jk rowling mary grandpr
2,Stephenie Meyer,2005,Twilight,3,twilight,stephenie meyer
3,Harper Lee,1960,To Kill a Mockingbird,4,kill mockingbird,harper lee
4,F. Scott Fitzgerald,1925,The Great Gatsby,4,great gatsby,f scott fitzgerald


## Content-based Recommender

In [152]:
import dill 
dill.dump_session('notebook_env.db')

KeyboardInterrupt: 

In [153]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

In [154]:
dill.load_session('notebook_env.db')

EOFError: Ran out of input

In [155]:
import numpy as np
import pandas as pd
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import KFold

# Ensure nltk data is downloaded (stopwords, punkt, wordnet)
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sanjeevani1109/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sanjeevani1109/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sanjeevani1109/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [156]:

def prepare_tfidf(df):
    # Combine title and author after cleaning for more context
    df['combined'] = df['cleaned_title'] + " " + df['cleaned_author']
    # df['combined'] = df['combined'].apply(clean_text)
    
    tfidf_vectorizer = TfidfVectorizer(max_features=500, stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(df['combined'])
    return tfidf_matrix, tfidf_vectorizer


In [ ]:
gbx.head()

,Author,Year,Title,Mean Rating,cleaned_title,cleaned_author
0,"J.K. Rowling, Mary GrandPrÃ©",1997.0,Harry Potter and the Philosopher's Stone,3,harry potter philosopher stone,jk rowling mary grandpr
1,Harper Lee,1960.0,To Kill a Mockingbird,4,kill mockingbird,harper lee
2,F. Scott Fitzgerald,1925.0,The Great Gatsby,4,great gatsby,f scott fitzgerald
3,J.R.R. Tolkien,1937.0,The Hobbit or There and Back Again,4,hobbit back,jrr tolkien
4,J.D. Salinger,1951.0,The Catcher in the Rye,4,catcher rye,jd salinger


In [ ]:
gbx['Title'].value_counts()

Title
Harry Potter and the Philosopher's Stone                                                            1
The Fourth Hand                                                                                     1
Rachael Ray 365: No Repeats--A Year of Deliciously Different Dinners (A 30-Minute Meal Cookbook)    1
á½ÏÎ­ÏÏÎµÎ¹Î±                                                                                   1
Manual do guerreiro da luz                                                                          1
                                                                                                   ..
The Elegant Universe: Superstrings, Hidden Dimensions, and the Quest for the Ultimate Theory        1
Seven Up                                                                                            1
Hard Eight                                                                                          1
What Is the What: The Autobiography of Valentino Achak Deng                 

In [ ]:
bx.head()

,Title,Author,Year,Scaled Rating,cleaned_title,cleaned_author
0,Classical Mythology,Mark P. O. Morford,2002,1,classical mythology,mark p morford
1,Clara Callan,Richard Bruce Wright,2001,3,clara callan,richard bruce wright
2,Decision in Normandy,Carlo D'Este,1991,3,decision normandy,carlo deste
3,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,3,flu story great influenza pandemic search viru...,gina bari kolata
4,The Mummies of Urumchi,E. J. W. Barber,1999,1,mummy urumchi,e j w barber


In [157]:
import random

def generate_recommendations(df, query, top_k=5, similarity_threshold=0.1):

    tfidf_matrix, tfidf_vectorizer = prepare_tfidf(df)
    
    #transforming the cleaned query to match TF-IDF dimensions
    query_vector = tfidf_vectorizer.transform([clean_text(query)])
    
    #computing cosine similarity
    cosine_sim = cosine_similarity(query_vector, tfidf_matrix).flatten()

    # Sorting items based on similarity scores, but only consider those above the threshold
    above_threshold_indices = np.where(cosine_sim >= similarity_threshold)[0]

    top_indices = np.argsort(cosine_sim[above_threshold_indices]) # Fetch more items for diversity
    
    #times 2 for safety factor
    random_selection = random.choices(top_indices, k=(len(top_indices) % top_k**2))
    
    # #fetching top indices 
    # top_indices = np.argsort(cosine_sim)[-top_k*10:]  #fetch more items for diversity (*10)
    
    seen = set()
    unique_recommendations = []
    # Reverse because when we sort we get smallest to largest
    print(len(random_selection))
    for index in random_selection:
        actual_index = above_threshold_indices[index]
        title_author = (df.iloc[actual_index]['Title'].lower(), df.iloc[actual_index]['Author'].lower(), df.iloc[actual_index]['Year'])

        same_as_query = df.iloc[actual_index]['Title'].lower() == query.lower()            

        if title_author not in seen and not same_as_query:
            seen.add(title_author)
            unique_recommendations.append(actual_index)

        # Continue adding until we truly have top_k unique or exhaust the list
        if len(unique_recommendations) == top_k or len(seen) == len(top_indices):
            break

        # if len(unique_recommendations) == top_k:
        #     break
    
    recommendations = df.iloc[unique_recommendations][['Title', 'Author', 'Year']]
    return recommendations  # Return recommendations in descending order of similarity
    # return recommendations



In [167]:
# df_to_use should be either your bx or gbx DataFrame
# Ensure 'Title' and 'Author' columns are present
query_title = "Love"
recommended_titles = generate_recommendations(gbx, query_title, top_k=5)
print(recommended_titles)

12
                                                  Title          Author  Year
8374                                    True Love Story    Willow Aster  2013
4008                                     Love, Stargirl  Jerry Spinelli  2007
6269                              Love in the Afternoon    Lisa Kleypas  2010
4873                                          Love wins        Rob Bell  2011
196   Marley & Me: Life and Love with the World's Wo...     John Grogan  2005


In [ ]:
bx.shape

(270151, 7)

In [ ]:
gbx.shape

(794, 7)

In [ ]:
# gbx[gbx['Title'].str.contains("Harry Potter", case=False)]
gbx[gbx['Title'].str.contains("Harry Potter")]['Title'].unique()


In [ ]:
gbx[gbx['Author'].str.contains("Dan Brown", case=False)]

In [ ]:
bx[bx['Title'].str.contains("Harry Potter")]['Title'].unique()
